### Prj06

Approximate the price $e^{-rT}\mathbb E [F(S)]$, where

- Asset follows $S = GBM(S_0, r, \sigma)$;
- Knock-in payoff is given as
$$F(S) = 1000 \cdot I(S_T > K) \cdot I\Big(\min_{1\le k \le m} S(t_k) < H\Big).$$

__Parameters__

- $r = 5\%, \sigma = 15\%, S(0) = 95$
- $T = 0.25, m = 50, H = 85, K = 96.$
- $k = 1000$, $n = 1000$

__Todo__

- Repeat $k$ times of the ordinary MC, with $n$ simulation for each MC. Find mean, MSE, and confidence interval using $k$ many MC outputs.

- Repeat the above procedure with importance sampling.

- Explain your decision of new probability in your importance sampling. Is it optimal choice?

### Answer for question 1

###### Knock-in payoff is given as below
$$F(S) = exp(-rT)\cdot 1000\cdot I(S_T > K)\cdot I(min_{1 \leq k \leq m} S (t_k) < H) $$
Suppose the underlying asset is modeled through a process of the form
$$ S(t_n) = S(0)\exp(L_n), L_n = \sum_{i=1}^n X_i, $$
with $X_1, X_2,$...i.i.d. and $L(0) = 0. $
The option payoff is
$$ \mathrm{Payoff} = 1000 \cdot I(S_T > K)I\Big(\min \limits_{1\leq k\leq m} S(t_k) < H\Big) $$
$$ \mathrm{Payoff} = 1000 \cdot I(L_T > c)I\Big(\min \limits_{1\leq k\leq m} L(t_k) < -b\Big) = 1000 \cdot I(L_m > c, \tau < m)$$
with $ 0 < t_1 < \cdots < t_m = T $, where $c = \log(K/S(0)), b = -\log(H/S(0)) $, $\tau$ is the first time the random walk $L_n$ drops below $-b$. If $b$ or $c$ is large, the probability of a payoff is small. To increase the probability of a payoff, we need to drive $L_n$ down toward $-b$ and then up toward $c$. 
Suppose $X_i$ have cumulant generating function $\phi$ and consider importance sampling estimators of the following form: exponentially twist the distribution of the $X_i$ by some $\theta_{-}$ until the barrier is crossed

In [1]:
import scipy.stats as ss
import numpy as np
import matplotlib.pyplot as plt

class GBM():
    def __init__(self, Drift, Vol, InitState):
        self.Drift = Drift #scalar
        self.Vol = Vol #scalar
        self.InitState = InitState
        self.Mu = lambda x, t: Drift * x
        self.Sigma = lambda x, t: Vol * x

    def _Wn(self,T,m):
        t = np.linspace(0,T,m+1)
        Wh = np.zeros(m+1)
        for i in range(m):
            DeltaW = np.sqrt(t[i+1] - t[i]) * np.random.normal()
            Wh[i+1] = Wh[i] + DeltaW
        return Wh

    def _explicit_sol_(self, t, W_t):
        x0 = self.InitState
        b = self.Drift
        sigma = self.Vol
        return x0 * np.exp((b - sigma**2/2.) * t + sigma * W_t)

In [2]:
## set the main function we will use to solve the question:

if __name__ == "__main__":
    r = 0.05
    vol = 0.15
    S0 = 95
    iGBM = GBM(r,vol,S0)
    T = 0.25
    m = 50
    H = 85
    K = 96
    k = 1000
    n = 1000

    Price = np.zeros(n)
    for j in range(n):
        payoff = 0
        for i in range(k):
            W_t = iGBM._Wn(T,m)
            t = np.linspace(0,T,m+1)
            St = iGBM._explicit_sol_(t,W_t)

            if St[-1] > K and min(St) < H:
                payoff = payoff + 1000
        Price[j] = payoff / k*np.exp(-r * T)

    print('The final price is ',np.mean(Price))

    mean = np.mean(Price)
    print('The mean is ',mean)

    mse = np.mean((Price - np.mean(Price)**2))
    print('The MSE is ',mse)
    print('The 95 percent confidence interval is between %4.3f and %4.3f' %(mean - 2*np.sqrt(mse), mean + 2*np.sqrt(mse)))

The final price is  0.5481056792741043
The mean is  0.5481056792741043
The MSE is  0.247685843621577
The 95 percent confidence interval is between -0.447 and 1.543


### Answer for question 2

### Project 06

###### Knock-in payoff is given as below

$$F(S) =exp(-rT) \cdot 1000\cdot I(S_T > K)\cdot I(min_{1 \leq k \leq m} S (t_k) < H) $$

###### Crude Monte Carlo simulation can be sampled from the below normal random distribution f(x)

$$ S_t = S_0 * exp(L_n)  $$
$$ L_n = \sum_{i=1}^{m}X_i  $$
$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t,\sigma^2 \delta t)  $$

###### If Importance Sampling (IS) is adopted, the alternative normal random distribution g(x) and the likelihood ratio are as follows

$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t-b,\sigma^2 \delta t)  $$
$$ \frac {f(x_1,x_2,...,x_m)}{g(x_1,x_2,...,x_m)} = exp(-\frac{1}{2}\sum_{k=1}^{m}{\frac{(x_k-\mu)}{\sigma \sqrt{\delta t}}}^2) \cdot exp(\frac{1}{2}\sum_{k=1}^{m}{\frac{(x_k-\mu+b)}{\sigma \sqrt{\delta t}}}^2)
= exp( \frac{b}{\sigma ^2 \delta t}\sum_{k=1}^{m}x_k-\frac{mb}{\sigma^2}(r-\frac{\sigma^2}{2})+\frac{mb^2}{2 \sigma^2 \delta t}     )                                                                            $$

###### The payoff is as below (The sampling should follow the alternative pdf g(x))

$$ exp(-rT)\cdot \frac {f(x_1,x_2,...,x_m)}{g(x_1,x_2,...,x_m)} \cdot F^{g(x)}(S)$$





In [3]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy import stats

#risk-free rate
r = 0.05
#volatility
vol = 0.15
#initial price
S0 = 95
#maturity
T = 0.25
#time steps
m = 50
#barrier price
H = 85
#maturity price
K = 96
#number of MC simulation
k = 1000
#simulation time for each MC
n = 1000

#time interval
dt=T/m

###### equivalent mu and sig
###### b value determined from the following equation
$$ b=\frac{1}{m} log(\frac{S_0}{H}) $$

In [4]:
b = math.log(S0/H)/m
c = math.log(K/S0)

mudt2 = (r - 0.5*vol**2)*dt - b
mudt1 = (r - 0.5*vol**2)*dt

sidt = vol*np.sqrt(dt)

NCrossed = np.zeros(k)

#Payoff=np.zeros(k)
Times = np.zeros(k)
StockVals = np.zeros(k)
ISRatio = np.zeros(k)

Ncrossed = 0
TBreach = 0
Price = np.zeros(n)

for counter in range(n):
    Payoff = np.zeros(k)
    for i in range(k):
        vetZ = np.random.normal(mudt2,sidt,m)
        ##vetZ2 = np.hstack()
        vetZ2 = np.hstack((math.log(S0),vetZ))
        LogPath = np.cumsum(vetZ2)
        St = np.exp(LogPath)


        jcross = 0
        for j in range(len(St)):
            if St[j] < H:
                jcross = j
                break
            #print(jcross)
            #the path crosses the lower barrier
        if jcross > 0:
                
            #TBreach = jcross - 1
            #Times[Ncrossed] = TBreach * dt
            #StockVals[Ncrossed] = St[jcross]
            #ISRatio[Ncrossed] = np.exp(TBreach*b**2/2/vol**2/dt+b/vol**2/dt*np.sum(vetZ[1:TBreach])-TBreach*b/vol**2*(r-vol**2/2))
            ISRatio[Ncrossed] = np.exp(m*b**2/2/vol**2/dt + b/vol**2/dt*np.sum(vetZ) - m*b/vol**2*(r-vol**2/2))
                
            if St[-1] > K:
                Payoff[Ncrossed] = ISRatio[Ncrossed] * 1000*np.exp(-r*T)
                Ncrossed = Ncrossed + 1
        
    Price[counter] = np.sum(Payoff)/k
    #print(counter)
print('The final payoff is: ',np.mean(Price))

The final payoff is:  0.5354847815486451


In [5]:
mean = np.mean(Price)
print('The mean is ',mean)
mse = np.mean((Price-np.mean(Price)**2))
print('The MSE is ',mse)
print('The 95 percent confidence interval is between %4.3f and %4.3f' %(mean - 2*np.sqrt(mse),mean + 2*np.sqrt(mse)))

The mean is  0.5354847815486451
The MSE is  0.2487408302784451
The 95 percent confidence interval is between -0.462 and 1.533


### Answer for question 3

The optimal biased density function should have a similar shape of the original density function but with a thicker tails, in other words, the biased density functio should have a revised drift value as well as an expanded standard deviation. 

In my selection of biased density function, I only assign an alternative dirft to inrease the chance of crossing the barrier. I assume the logarithmic proportion is distributed evenly along the whole time steps. Although the chance of crossing the barier increases. The change of crossing the exercise price K at maturity date has decreased. So my selection of biased density function is not the optimal.

###### Revised drift value
$$ b=\frac{1}{m} log(\frac{S_0}{H}) $$
###### Initial Density Function f(x)
$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t,\sigma^2 \delta t)  $$
###### Biased Density Function g(x)
$$ X_i \backsim N((r-\frac{\sigma^2}{2})\delta t-b,\sigma^2 \delta t)  $$